In [1]:
import torch
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset , DataLoader
import zipfile
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import csv

In [2]:
#with zipfile.ZipFile("archive.zip" ,"r") as zip_ref:
    #zip_ref.extractall("dataset")

In [3]:
image_train = r"C:\Users\ASUS\OneDrive\Masaüstü\ml_traffic_sign\dataset\Train"

for number , class_train in enumerate(os.listdir(image_train)):
    print(number , class_train)
    

FileNotFoundError: [WinError 3] Sistem belirtilen yolu bulamıyor: 'C:\\Users\\ASUS\\OneDrive\\Masaüstü\\ml_traffic_sign\\dataset\\Train'

In [4]:
csv_path = r"C:\Users\ASUS\OneDrive\Masaüstü\ml_traffic_sign\dataset\signnames.csv"

with open(csv_path, newline='', encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        print(row["ClassId"], "→", row["SignName"])


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ASUS\\OneDrive\\Masaüstü\\ml_traffic_sign\\dataset\\signnames.csv'

In [13]:
class ImageDataset(Dataset):
    def __init__(self ,image_train , transform):
        self.image_train = image_train
        self.image_paths = []
        self.labels = []
        self.class_name = {}
        self.transform = transform
        self.id_to_name = None

        csv_path = r"C:\Users\ASUS\OneDrive\Masaüstü\traffic-sign-classifier\dataset\signnames.csv"
        names_df = pd.read_csv(csv_path)
        self.id_to_name = dict(zip(names_df["ClassId"], names_df["SignName"]))
            
        for class_id in sorted(os.listdir(image_train), key=lambda x: int(x)):
            class_id = int(class_id)
            self.class_name[class_id] = self.id_to_name[class_id]
            class_path = os.path.join(image_train, str(class_id))
        
            for img_name in os.listdir(class_path):
                self.image_paths.append(os.path.join(class_path , img_name))
                self.labels.append(class_id)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        

        if self.transform:
            image = self.transform(image)

        return image , label 

In [14]:
class TestImageDataset(Dataset):
    def __init__(self, test_csv_path, base_folder, transform=None):
        self.transform = transform
        self.base_folder = base_folder
        self.image_paths = []
        self.labels = []
        test_df = pd.read_csv(test_csv_path)
        for index, row in test_df.iterrows():
            image_name = row['Path'].split('/')[-1]
            image_path = os.path.join(self.base_folder, image_name)
            
            self.image_paths.append(image_path)
            self.labels.append(row['ClassId'])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        

        if self.transform:
            image = self.transform(image)

        return image, label

In [15]:
transform = transforms.Compose([

    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.5,0.5,0.5] , std = [0.5,0.5,0.5])
])

In [17]:
image_train = r"C:\Users\ASUS\OneDrive\Masaüstü\traffic-sign-classifier\dataset\Train"
image_test = r"C:\Users\ASUS\OneDrive\Masaüstü\traffic-sign-classifier\dataset\Test"

image_train_dataset = ImageDataset (image_train , transform = transform)
image_test_dataset = TestImageDataset(r"C:\Users\ASUS\OneDrive\Masaüstü\traffic-sign-classifier\dataset\Test.csv",image_test , transform = transform)


In [18]:
train_image_loader = DataLoader(dataset= image_train_dataset , batch_size = 32 , shuffle = True, num_workers=0)
test_image_loader = DataLoader(dataset= image_test_dataset , batch_size = 32 , shuffle = False, num_workers=0)

In [19]:
for images, labels in train_image_loader:
    print(images.shape, labels.shape)

    img = images[0].numpy()
    print(img.shape)

    label = labels[0].item()
    print(image_train_dataset.class_name[label])

    img = np.transpose(img, (1, 2, 0))
    print(img.shape)
    print(label)
    break


torch.Size([32, 3, 64, 64]) torch.Size([32])
(3, 64, 64)
Speed limit (50km/h)
(64, 64, 3)
2


Custom CNN Architecture

In [20]:
import torch.nn as nn
import torch.optim as optim


In [21]:
class CustomCnnModel(nn.Module):
    def __init__(self , input_dim, num_classes):
        super(CustomCnnModel,self).__init__()
        self.input_dim = input_dim
        self.num_classes = num_classes

        self.conv_layers = nn.Sequential(

            #FC1
            nn.Conv2d(3 ,32 , kernel_size=3 , stride=1 , padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2 ,stride=2),

            #FC2
            nn.Conv2d(32 ,64 , kernel_size=3 , stride=1 , padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2 ,stride=2),

            #FC3
            nn.Conv2d(64 ,128 , kernel_size=3 , stride=1 , padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2 ,stride=2),

            #FC4
            nn.Conv2d(128 ,256 , kernel_size=3 , stride=1 , padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2 ,stride=2)

        )

        self._to_linear = None
        self._get_conv_output(self.input_dim)


        self.fc_layers = nn.Sequential(
            nn.Linear(self._to_linear ,512),
            nn.ReLU(),
            nn.Linear(512,128),
            nn.ReLU(),
            nn.Linear(128,self.num_classes),


        )
    def _get_conv_output(self, input_dim=64):
        with torch.no_grad():
            dummy_input = torch.zeros(1,3 ,input_dim , input_dim)
            output = self.conv_layers(dummy_input)
            self._to_linear  = output.view(1 , -1).size(1)
           
    
    
    
    def forward(self,x):
        x = self.conv_layers(x)
        x = x.view(x.size(0) , -1)
        x = self.fc_layers(x)
        return x

In [22]:
#Initialize Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model =CustomCnnModel(input_dim=64 , num_classes=43).to(device)


In [23]:
print(model)

CustomCnnModel(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [39]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters() , lr=0.001)

In [42]:
# Training loop

epochs = 20
for epoch in range (epochs):
    model.train()
    running_loss = 0.0
    for images , labels in train_image_loader:
        images , labels = images.to(device) , labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs , labels)
        loss.backward()
        optimizer.step()
    running_loss+= loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss :{running_loss/len(train_image_loader)}")

Epoch 1/20, Loss :0.0027158347856356697
Epoch 2/20, Loss :0.0029359486130283476
Epoch 3/20, Loss :0.0028563185578263798
Epoch 4/20, Loss :0.0030248229305273556
Epoch 5/20, Loss :0.002645829956838785
Epoch 6/20, Loss :0.002885574423275025
Epoch 7/20, Loss :0.0028104307600957923
Epoch 8/20, Loss :0.0026011793687331928
Epoch 9/20, Loss :0.0026105366173226137
Epoch 10/20, Loss :0.0028680448236512125
Epoch 11/20, Loss :0.002995033832008648
Epoch 12/20, Loss :0.0027484399848427704
Epoch 13/20, Loss :0.002788932249557719
Epoch 14/20, Loss :0.002972953665703778
Epoch 15/20, Loss :0.002865532483012213
Epoch 16/20, Loss :0.0028002487699125955
Epoch 17/20, Loss :0.002698509961512505
Epoch 18/20, Loss :0.002725444454547826
Epoch 19/20, Loss :0.003152967666140583
Epoch 20/20, Loss :0.002642994020346911
